In [50]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import cloudscraper
from datetime import datetime
pd.options.mode.chained_assignment = None 
import re

In [51]:
df_raw = pd.read_csv('DA_postings.csv')

In [52]:
#df_salary = df_raw[df_raw['Salary'].notna()]
df_salary = df_raw.copy()
df_salary.head()

,Job_title,Company,Rating,Location,Salary,Date_Posted,Date_Extracted,Easy_Apply,Job_Summary,Job_url
0,"Business Intelligence (BI) Developer/Analyst, IT",Canaccord Genuity Corp.,3.7,"Toronto, ON",NaN,Posted 30+ days ago,2023-03-16,No,4+ years of experience with SQL/data manipulat...,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
1,Intelligence Analyst,Communications Security Establishment,4.1,"Toronto, ON","$74,823–$97,710 a year",Posted 30+ days ago,2023-03-16,No,Experience conducting research and analyzing i...,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,Junior Logistics Data Analyst,Atlantic Packaging Products Ltd,3.1,"Remote in Scarborough, ON",NaN,Posted 30+ days ago,2023-03-16,Easily apply,Advanced knowledge of data Analytics in Excel....,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
3,Data Analyst,SickKids,4.2,"Toronto, ON",NaN,Posted 30+ days ago,2023-03-16,No,Assist end users with data deposition and data...,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
4,DATA & SCHEDULING ANALYST,City of Toronto,4.1,"Toronto, ON",$45.08–$49.41 an hour,Posted 30+ days ago,2023-03-16,No,Queries and analyzes data for performance meas...,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...


In [57]:
#df_salary['Salary_Amount'] = df_salary['Salary'].apply(lambda x: " ".join(x.split(' ',-2)[:1]))

In [35]:
#df_salary['Salary_Unit'] = df_salary['Salary'].apply(lambda x: " ".join(x.split(' ', -2)[1:]))

In [54]:
df = df_salary.copy()

### Parse salary string data and change to float data type

In [55]:
# Function to parse salary
def parse_salary(salary):
    if pd.isnull(salary):
        return [np.nan, np.nan, np.nan]

    # Check if salary contains a range
    if '-' in salary:
        pattern = r'([\d,]+)-([\d,]+)\s*(?:a|an)\s*(\w+)'
        matches = re.search(pattern, salary)
        if matches:
            lower = float(matches.group(1).replace(",", ""))
            upper = float(matches.group(2).replace(",", ""))
            unit = matches.group(3)
        else:
            return [np.nan, np.nan, np.nan]
    else:
        pattern = r'([\d,]+)\s*(?:a|an)\s*(\w+)'
        matches = re.search(pattern, salary)
        if matches:
            lower = float(matches.group(1).replace(",", ""))
            upper = lower
            unit = matches.group(2)
        else:
            return [np.nan, np.nan, np.nan]
    
    return [lower, upper, unit]

# Apply function to Salary column and create a new DataFrame
df[['Salary_Lower', 'Salary_Upper', 'Salary_Unit']] = df['Salary'].apply(parse_salary).apply(pd.Series)


In [58]:
#df[df['Salary_Unit']=='n']

### Pull Days since job was posted from the extracted date

In [41]:
def extract_days(s):
    pattern = r'(\d+|\d+\+)'  # matches digits or digits followed by '+'
    match = re.search(pattern, s)
    if match:
        # if '30+' is found in the string, return 31
        if '+' in match.group(0):
            return 31
        else:
            return int(match.group(0))
    else:
        return np.nan  # return NaN if no match is found

# create new column 'Days_Since_Posted' by applying the function to the appropriate column
df['Days_Since_Posted'] = df['Date_Posted'].apply(extract_days)


In [47]:
df_final = df[['Job_title', 'Company', 'Rating', 'Location', 'Days_Since_Posted', 'Date_Extracted', 'Easy_Apply', 'Salary', 'Salary_Lower','Salary_Upper', 'Salary_Unit', 'Job_Summary', 'Job_url']]

In [48]:
df_final

,Job_title,Company,Rating,Location,Days_Since_Posted,Date_Extracted,Easy_Apply,Salary,Salary_Lower,Salary_Upper,Salary_Unit,Job_Summary,Job_url
1,Intelligence Analyst,Communications Security Establishment,4.1,"Toronto, ON",30,2023-03-16,No,"$74,823–$97,710 a year",97710.0,97710.0,year,Experience conducting research and analyzing i...,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
4,DATA & SCHEDULING ANALYST,City of Toronto,4.1,"Toronto, ON",30,2023-03-16,No,$45.08–$49.41 an hour,41.0,41.0,n,Queries and analyzes data for performance meas...,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
5,Senior Business Analyst,Embross North America,3.3,"Richmond Hill, ON",30,2023-03-16,Easily apply,"$80,000–$95,000 a year",95000.0,95000.0,year,We are looking for a experienced Business Anal...,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
6,Bilingual Business Insight Program Analyst - R...,NTT Business Process Outsourcing,2.9,"Remote in Etobicoke, ON",30,2023-03-16,Easily apply,"$45,000–$60,000 a year",60000.0,60000.0,year,Participate in the preparation of data for int...,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
9,Data Analyst,Sun Life,3.9,"Hybrid remote in Toronto, ON",30,2023-03-16,No,"$51,100 a year",51100.0,51100.0,year,Create data tools and optimize data pipelines ...,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,Credit Analyst - 12 month contract,Vaco Lannick,3.6,"Vaughan, ON",30,2023-03-16,No,"$55,000 a year",55000.0,55000.0,year,Demonstrates skill in data analysis techniques...,https://ca.indeed.com/rc/clk?jk=9289714f725293...
743,Credit Analyst - 12 month contract,Vaco Lannick,3.6,"Vaughan, ON",30,2023-03-16,No,"$55,000 a year",55000.0,55000.0,year,Demonstrates skill in data analysis techniques...,https://ca.indeed.com/rc/clk?jk=62e4687f3c912d...
758,Credit Analyst - 12 month contract,Vaco Lannick,3.6,"Vaughan, ON",30,2023-03-16,No,"$55,000 a year",55000.0,55000.0,year,Demonstrates skill in data analysis techniques...,https://ca.indeed.com/rc/clk?jk=428687877e6e18...
773,Credit Analyst - 12 month contract,Vaco Lannick,3.6,"Vaughan, ON",20,2023-03-16,No,"$55,000 a year",55000.0,55000.0,year,Demonstrates skill in data analysis techniques...,https://ca.indeed.com/rc/clk?jk=60cc3dd037ec8b...
